<a href="https://colab.research.google.com/github/tophercollins/dnd-statblock-generator/blob/main/dnd_statblock_generator_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Statblock Generator for DND 5e using Fine-Tuning with LLama 3 8B

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("tophercollins/dnd_monster_sample", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.232900
2,2.283100
3,2.251900
4,2.045000
5,1.789300
6,1.576400
7,1.295300
8,1.037500
9,0.917700
10,0.816700



<a name="Inference"></a>
# Inference

### 1. Basic request with just a name or idea.

**1a.**
Input: Make a statblock for a cat called Mittens.

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... Make a statblock for a cat called Mittens.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... Make a statblock for a cat called Mittens.

### Input:


### Response:
Name:Cat, Mittens
Size:Medium
Type:Fiend (Yugoloth)
Alignment:NE
AC:12
HP:7
Speeds:30
STR:8
DEX:12
CON:10
INT:3
WIS:10
CHA:5
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:Ignan, Terran
Critical Rating:0.0<|end_of_text|>


**1b.**
Input: Create a Fire Demon, give it an interesting name.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... Create a statblock for a Fire Demon, give it an interesting name.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... Create a statblock for a Fire Demon, give it an interesting name.

### Input:


### Response:
Name:Fire Demon
Size:Medium
Type:Fiend (Demon)
Alignment:CE
AC:13
HP:33
Speeds:30
STR:14
DEX:8
CON:13
INT:5
WIS:8
CHA:5
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:Abyssal
Critical Rating:1.0<|end_of_text|>


### 2. Request with specific stat information.

**2a.**
Input: Give me a statblock for a Fey creature, with speed 45, size small, and critical rating 2.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... Give me a statblock for a Fey creature, with speed 45, size small, and critical rating 2.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... Give me a statblock for a Fey creature, with speed 45, size small, and critical rating 2.

### Input:


### Response:
Name:Fey creature, statblock
Size:small
Type:Fey
Alignment:NE
AC:12
HP:22
Speeds:30, fly 30
STR:8
DEX:14
CON:10
INT:8
WIS:11
CHA:10
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:Common, Elvish
Critical Rating:2<|end_of_text|>


**2b.**
Input: Make me an alien blob with over 100 Hit Points, that can speak telepathically.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... Make me an alien blob with over 100 Hit Points, that can speak telepathically.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... Make me an alien blob with over 100 Hit Points, that can speak telepathically.

### Input:


### Response:
Name:Alien Blob
Size:Huge
Type:Alien
Alignment:NE
AC:13
HP:101
Speeds:30
STR:19
DEX:10
CON:17
INT:10
WIS:10
CHA:10
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:Telepathy 60
Critical Rating:5.0<|end_of_text|>


# Test it out

**1a.** What do you want to make?

In [ ]:
your_prompt1 = input('What do you want to make? ')

What do you want to make? A dinosaur, with bloody head. Give it a funky name.


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... " + your_prompt1, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... A dinosaur, with bloody head. Give it a funky name.

### Input:


### Response:
Name:Funky Dinosaur
Size:Medium
Type:Monstrosity
Alignment:U
AC:13
HP:34
Speeds:30
STR:15
DEX:10
CON:14
INT:3
WIS:10
CHA:4
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:nan
Critical Rating:1.0<|end_of_text|>


**1b.** Try again?

In [ ]:
your_prompt = input('What do you want to make? ')

What do you want to make? A film noir supervillain, femme fatale. Come up with an appropriate name.


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... " + your_prompt, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... A film noir supervillain, femme fatale. Come up with an appropriate name.

### Input:


### Response:
Name:Laurel Fortune
Size:Medium
Type:Humanoid
Alignment:CE
AC:12
HP:45
Speeds:30
STR:11
DEX:15
CON:10
INT:14
WIS:12
CHA:16
Saving Throws:DEX, CHA
Skills:Deception, Insight, Perception, Persuasion
Senses:Darkvision 60
Languages:Any
Critical Rating:6.0<|end_of_text|>


**1c.** Once more for luck?

In [ ]:
your_prompt3 = input('What do you want to make? ')

What do you want to make? A floppy disk monster.


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a statblock... " + your_prompt3, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a statblock... A floppy disk monster.

### Input:


### Response:
Name:Floppy Disk Monster
Size:Medium
Type:Construct
Alignment:NE
AC:13
HP:33
Speeds:30
STR:14
DEX:10
CON:13
INT:8
WIS:8
CHA:10
Saving Throws:nan
Skills:nan
Senses:Darkvision 60
Languages:Construct
Critical Rating:1.0<|end_of_text|>


# Saving Model

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# model.save_pretrained("dnd_statblock_generator_lora_model") # Local saving
# tokenizer.save_pretrained("dnd_statblock_generator_lora_model") # Local saving
# model.push_to_hub("tophercollins/dnd_statblock_generator_lora_model") # Online saving
#btokenizer.push_to_hub("tophercollins/dnd_statblock_generator_lora_model") # Online saving